In [1]:
import csv
import pandas as pd
import os
import numpy as np
import pyodbc
import sqlalchemy as db
import urllib

dataframe = pd.read_csv("Data_Agregate_Octav.csv")

print("df loaded")

df loaded


C:\Users\Sky\AppData\Local\Temp\ipykernel_19348\3191462434.py:9: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv("Data_Agregate_Octav.csv")


In [2]:
# Setare tip date pt fiecare coloana, fillna
dataframe['Cod unic candidat'] = dataframe['Cod unic candidat'].astype(str)
dataframe['Sex'] = dataframe['Sex'].astype(str)
dataframe['Specializare'] = dataframe['Specializare'].astype(str)
dataframe['Profil'] = dataframe['Profil'].astype(str)
dataframe['Fileira'] = dataframe['Fileira'].astype(str)
dataframe['Forma de învățământ'] = dataframe['Forma de învățământ'].astype(str)
dataframe['Mediu candidat'] = dataframe['Mediu candidat'].astype(str)
dataframe['Unitate (SIIIR)'] = dataframe['Unitate (SIIIR)'].astype(str)
dataframe['STATUS'] = dataframe['STATUS'].astype(str)
dataframe['Medie'] = dataframe['Medie'].astype(float)
dataframe['longName'] = dataframe['longName'].astype(str)
dataframe['schoolType'] = dataframe['schoolType'].astype(str)
dataframe['statut'] = dataframe['statut'].astype(str)
dataframe['propertyForm'] = dataframe['propertyForm'].astype(str)
dataframe['fundingForm'] = dataframe['fundingForm'].astype(str)
dataframe['county'] = dataframe['county'].astype(str)
dataframe['locality'] = dataframe['locality'].astype(str)
dataframe['runningWater'] = dataframe['runningWater'].fillna(0)
dataframe['canalization'] = dataframe['canalization'].fillna(0)
dataframe['gas'] = dataframe['gas'].fillna(0)
dataframe['electricity'] = dataframe['electricity'].fillna(0)
dataframe['heating'] = dataframe['heating'].fillna(0)
dataframe['salubrity'] = dataframe['salubrity'].fillna(0)
dataframe['centralHeating'] = dataframe['centralHeating'].fillna(0)


# Inlocuit nan din medie cu 0
dataframe['Medie'] = dataframe['Medie'].fillna(0)

print('Data type check done. Nans filled.')

Data type check done. Nans filled.


# Uniformizarea font size-ului cu upper,Inlocuirea diacriticelor in coloane, inlocuirea semnelor speciale, inlocuirea "DA" si "NU" cu 1 & 0s


In [3]:
# Inlocuirea diacriticelor si a semnelor speciale in coloane
diacr = {"Ă":"A","Ș":"S","Ş":"S","Ț":"T","Ţ":"T","Â":"A","Î":"I",'"':"", ",":"","-":"_", " ":"_", "-":"_", "/":"_"}
# Create a translation 
translation_table = str.maketrans(diacr)

# Apply the translation to the entire df
try:
    for x in dataframe:
        if dataframe[x].dtypes == 'object':
            dataframe[x] = dataframe[x].astype(str).str.upper()
            dataframe[x] = dataframe[x].astype(str).str.translate(translation_table)
            dataframe[x] = dataframe[x].replace("__", "_", regex = True)
            dataframe[x] = dataframe[x].replace(to_replace=['NU', 'DA'], value=["0", "1"])
            dataframe[x] = dataframe[x].replace(to_replace=['IN_CURS'], value=["1"])
    
except Exception as exc:
    print(x, " ", exc)
    pass

print("Done")

Done


# Split coloana "toilet" si generarea a 2 noi coloane

In [4]:
# Split coloana "toilet" in doua coloane cu valori categorice
dataframe.toilet.unique()

amplasare_toaleta = [(0 if "EXTERIORUL" in str(x) else (1 if "INTERIORUL" in str(x) else 0)) for x in dataframe['toilet']]
stare_toaleta = [(1 if "CORESPUNZ" in str(x) else (0 if "NECORESPUNZ" in str(x) else (1 if "REABILITARE" in str(x) else 0))) for x in dataframe['toilet']]

# Alipire la df
dataframe["amplasare_toaleta_0ext_1int_0nan"] = amplasare_toaleta
dataframe["stare_toaleta_1coresp_0necoresp_1reabil_0nan"] = stare_toaleta

# Double-check type pt coloanele cu tipuri de date schimbate
dataframe['runningWater'] = dataframe['runningWater'].astype(int)
dataframe['canalization'] = dataframe['canalization'].astype(int)
dataframe['gas'] = dataframe['gas'].astype(int)
dataframe['electricity'] = dataframe['electricity'].astype(int)
dataframe['heating'] = dataframe['heating'].astype(int)
dataframe['salubrity'] = dataframe['salubrity'].astype(int)
dataframe['centralHeating'] = dataframe['centralHeating'].astype(int)
dataframe["amplasare_toaleta_0ext_1int_0nan"]= dataframe["amplasare_toaleta_0ext_1int_0nan"].astype(int)  
dataframe["stare_toaleta_1coresp_0necoresp_1reabil_0nan"] = dataframe["stare_toaleta_1coresp_0necoresp_1reabil_0nan"].astype(int)

dataframe.head(10)

,Unnamed: 0,Cod unic candidat,Sex,Specializare,Profil,Fileira,Forma de învățământ,Mediu candidat,Unitate (SIIIR),Unitate (SIRUES),...,canalization,gas,electricity,heating,salubrity,centralHeating,toilet,dateFrom,amplasare_toaleta_0ext_1int_0nan,stare_toaleta_1coresp_0necoresp_1reabil_0nan
0,0,10005,M,TEHNICIAN_IN_AUTOMATIZARI,TEHNIC,TEHNOLOGICA,ZI,URBAN,3261101959,320960511,...,1,1,1,1,1,1,IN_INTERIORUL_CLADIRII_STARE_CORESPUNZATOARE,11_02_2015_00:00,1,1
1,1,12037,M,STIINTE_SOCIALE,UMAN,TEORETICA,FRECVENTA_REDUSA,URBAN,3661100016,718401,...,0,0,0,0,0,0,NAN,NAN,0,0
2,2,12284,M,TEHNICIAN_IN_TURISM,SERVICII,TEHNOLOGICA,ZI,URBAN,1161104376,424268,...,0,0,0,0,0,0,NAN,NAN,0,0
3,3,12865,F,FILOLOGIE,UMAN,TEORETICA,ZI,URBAN,961100082,404517,...,1,1,1,0,1,1,IN_INTERIORUL_CLADIRII_STARE_CORESPUNZATOARE,29_12_2020_00:00,1,1
4,4,12923,F,ORGANIZATOR_BANQUETING,SERVICII,TEHNOLOGICA,ZI,RURAL,2461103282,1182013,...,0,0,0,0,0,0,NAN,NAN,0,0
5,5,13353,F,TEHNICIAN_IN_INDUSTRIA_TEXTILA,TEHNIC,TEHNOLOGICA,ZI,RURAL,661101499,831851,...,0,0,1,0,1,0,IN_EXTERIORUL_CLADIRII_STARE_CORESPUNZATOARE,12_02_2015_00:00,0,1
6,6,13302,M,TEHNICIAN_IN_MORARIT_PANIFICATIE_SI_PRODUSE_FA...,RESURSE_NATURALE_SI_PROTECTIA_MEDIULUI,TEHNOLOGICA,ZI,URBAN,3261100203,321182491,...,1,1,1,0,1,1,IN_INTERIORUL_CLADIRII_STARE_CORESPUNZATOARE,11_02_2015_00:00,1,1
7,7,14454,M,TEHNICIAN_DE_TELECOMUNICATII,TEHNIC,TEHNOLOGICA,ZI,URBAN,4061102264,223729,...,0,0,0,0,0,0,NAN,NAN,0,0
8,8,15268,F,TEHNICIAN_CHIMIST_DE_LABORATOR,RESURSE_NATURALE_SI_PROTECTIA_MEDIULUI,TEHNOLOGICA,ZI,RURAL,1561101625,803567,...,1,1,1,0,1,1,IN_INTERIORUL_CLADIRII_STARE_CORESPUNZATOARE,31_08_2020_00:00,1,1
9,9,14966,F,TEHNICIAN_DE_TELECOMUNICATII,TEHNIC,TEHNOLOGICA,ZI,URBAN,4061102264,223729,...,0,0,0,0,0,0,NAN,NAN,0,0


# Drop columns

In [5]:
columns_to_drop =  ['Unnamed: 0','Clasa', 'Subiect eb', 'Subiect ea', 'Limba modernă', 'Subiect ec', 'Subiect ed', 'Promoție', 'NOTE_RECUN_A',
                   'NOTE_RECUN_B', 'NOTE_RECUN_C', 'NOTE_RECUN_D', 'NOTE_RECUN_EA', 'NOTE_RECUN_EB', 'NOTE_RECUN_EC', 'NOTE_RECUN_ED',
                   'STATUS_A', 'STATUS_B', 'STATUS_C', 'STATUS_D', 'STATUS_EA', 'STATUS_EB', 'STATUS_EC', 'STATUS_ED', 'ITA',
                   'SCRIS_ITC', 'SCRIS_PMS', 'ORAL_PMO', 'ORAL_IO', 'NOTA_EA', 'NOTA_EB', 'NOTA_EC', 'NOTA_ED', 'CONTESTATIE_EA',
                   'NOTA_CONTESTATIE_EA', 'CONTESTATIE_EB', 'NOTA_CONTESTATIE_EB', 'CONTESTATIE_EC', 'NOTA_CONTESTATIE_EC',
                   'CONTESTATIE_ED', 'NOTA_CONTESTATIE_ED', 'PUNCTAJ DIGITALE', 'idParentSchool','schoolSocialLinks', 'schoolYearDescription',
                   'shortName', 'isPj', 'fiscalCode', 'street', 'streetNumber', 'postalCode',
                   'phoneNumber', 'faxNumber', 'email', 'id', 'buildingDescription','dateFrom','toilet', 'Unitate (SIRUES)','internalIdSchool','siruesCode','code','operatingMode']

dataframe = dataframe.drop(columns=columns_to_drop)

filename = 'Date_curatate_Calin.csv'
dataframe.to_csv(filename, encoding = "utf-8", index=True)

df1 = pd.read_csv(filename)
df1.head(10)

,Unnamed: 0,Cod unic candidat,Sex,Specializare,Profil,Fileira,Forma de învățământ,Mediu candidat,Unitate (SIIIR),STATUS,...,locality,runningWater,canalization,gas,electricity,heating,salubrity,centralHeating,amplasare_toaleta_0ext_1int_0nan,stare_toaleta_1coresp_0necoresp_1reabil_0nan
0,0,10005,M,TEHNICIAN_IN_AUTOMATIZARI,TEHNIC,TEHNOLOGICA,ZI,URBAN,3261101959,NEPROMOVAT,...,SIBIU,1,1,1,1,1,1,1,1,1
1,1,12037,M,STIINTE_SOCIALE,UMAN,TEORETICA,FRECVENTA_REDUSA,URBAN,3661100016,PROMOVAT,...,TULCEA,0,0,0,0,0,0,0,0,0
2,2,12284,M,TEHNICIAN_IN_TURISM,SERVICII,TEHNOLOGICA,ZI,URBAN,1161104376,PROMOVAT,...,BAILE_HERCULANE,0,0,0,0,0,0,0,0,0
3,3,12865,F,FILOLOGIE,UMAN,TEORETICA,ZI,URBAN,961100082,NEPROMOVAT,...,FAUREI,1,1,1,1,0,1,1,1,1
4,4,12923,F,ORGANIZATOR_BANQUETING,SERVICII,TEHNOLOGICA,ZI,RURAL,2461103282,PROMOVAT,...,SIGHETU_MARMATIEI,0,0,0,0,0,0,0,0,0
5,5,13353,F,TEHNICIAN_IN_INDUSTRIA_TEXTILA,TEHNIC,TEHNOLOGICA,ZI,RURAL,661101499,NEPROMOVAT,...,BECLEAN,0,0,0,1,0,1,0,0,1
6,6,13302,M,TEHNICIAN_IN_MORARIT_PANIFICATIE_SI_PRODUSE_FA...,RESURSE_NATURALE_SI_PROTECTIA_MEDIULUI,TEHNOLOGICA,ZI,URBAN,3261100203,NEPROMOVAT,...,CISNADIE,1,1,1,1,0,1,1,1,1
7,7,14454,M,TEHNICIAN_DE_TELECOMUNICATII,TEHNIC,TEHNOLOGICA,ZI,URBAN,4061102264,NEPROMOVAT,...,BUCURESTI_SECTORUL_6,0,0,0,0,0,0,0,0,0
8,8,15268,F,TEHNICIAN_CHIMIST_DE_LABORATOR,RESURSE_NATURALE_SI_PROTECTIA_MEDIULUI,TEHNOLOGICA,ZI,RURAL,1561101625,PROMOVAT,...,TITU,1,1,1,1,0,1,1,1,1
9,9,14966,F,TEHNICIAN_DE_TELECOMUNICATII,TEHNIC,TEHNOLOGICA,ZI,URBAN,4061102264,NEPROMOVAT,...,BUCURESTI_SECTORUL_6,0,0,0,0,0,0,0,0,0


# Uploadarea tabelului in SQLul din Azure (cod curs)

In [6]:
# Azure update with course details & credentials
database_name = "AID2023"
driver = '{ODBC Driver 18 for SQL Server}'
server_name = "aid-2023-srv.database.windows.net"

print("Connection details inputted.")

# Create the server url
server = 'tcp:{server_name},1433'.format(server_name = server_name)

#create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
    driver = driver,
    server=server,
    database=database_name,
    username="aid2023adminlog",
    password="@Aidr2023sql"
)                              

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
cursor: pyodbc.Cursor = cnxn.cursor() 

#execute a query with the cursor to check if the connection is maade
print("Connection is open") if cursor.execute("SELECT 1") else print(f"Connection is closed: {e}")

#obtain the parameters from the pyodbc connection string
params = urllib.parse.quote_plus(connection_string)
#create the engine
sqlalchemy_engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
print("dfs loaded")

try:
    df1.to_sql("Grupa3", sqlalchemy_engine, if_exists='replace',index = True)

except Exception as exc:
    print("Smth went wrong: ", exc)

print("done")

Connection details inputted.
Connection is open
dfs loaded
done
